









#Mount the google drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Install pre-requisites

In [ ]:
'''These lines install the necessary libraries for your task.

!pip install datasets installs the datasets library which is used for loading and processing datasets.
!pip install accelerate -U installs or upgrades the accelerate library. This library helps with training and using PyTorch models.
!pip install transformers installs the transformers library. This is a library for natural language processing (NLP) tasks. It provides pre-trained models like BART that we have used in the code.
'''

!pip install datasets
!pip install accelerate -U
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


#Load the dataset and split


In [ ]:
'''
This line uses the glob library to find all the .txt files in the specified directory.

glob.glob() is a function that returns a list of files matching a given pattern.
path + "/*.txt" creates a pattern that matches any file ending with .txt in the directory specified by path.
This line essentially creates a list of all text files found in the directory.
'''
import glob

def load_data(data):
  #Path to the summary
  path="/content/drive/MyDrive/IN-Abs/train-data/summary"

  #Get all the txt files with summaries
  all_files = glob.glob(path + "/*.txt")

  summary = []

  #Get the summaries
  for filename in all_files:
      with open(filename, 'r') as f:
          a = f.read()
          summary.append(a)

  #Path to the case files
  path = "/content/drive/MyDrive/IN-Abs/train-data/judgement"

  #Get all the txt files with case details
  all_files = glob.glob(path + "/*.txt")

  source = []

  for filename in all_files:
      with open(filename, 'r') as f:
          a = f.read()
          source.append(a)
          #Prepare the dataset

  for src,summ in zip(source,summary):
      data.append({"source" : src , "summary" : summ})

  #Convert the data from "dict" to "DatasetDict"
  # Reference : https://discuss.huggingface.co/t/convert-a-list-of-dictionaries-to-hugging-face-dataset-object/14670
  #from datasets import Dataset
  #data=Dataset.from_list(data)

  return data

In [ ]:
#Load the data
data=list()
data=load_data(data)

In [ ]:
data[0]["source"]

'Appeal No. 53 of 1969.\nAppeal by special leave from the judgment and decree dated November 1, 1968 of the Rajasthan High Court in Civil Regular Second Appeal No. 487 of 1962.\nC. B. Agarwala, Rameshwar Nath and Mahinder Narain, for the appellant.\nB. R. L. Iyengar, section K. Mehta and K. L. Mehta, for the respondents.\nThe Judgment of the Court was, delivered by Bachawat, J.\nThis appeal arises out of a suit for ejectment by a landlord against a tenant.\nThe defendant is the tenant of six shops belonging to Thakurji Shri Shri Dwarkadheeshji installed in the temple at Chaura Raasta, Jaipur.\nDevendra Prasad is the adhikari or manager of the temple.\nHe gave a notice to the defendant to quit the shop on August 1, 1957.\nOn February 28, 1958, the deity and Devendra Prasad filed a suit against the defendant claiming recovery of possession of the six shops and Rs. 1,006/ on account of arrears of rent.\nThe suit was governed by the Rajasthan Premises (Control of Rent and Eviction) Act, 19

In [ ]:
data[0]["summary"]

"Section 10(1) of the , does not confer on the appropriate Government the power to cancel or supersede a reference made thereunder in respect of an industrial dispute pending adjudication by the tribunal constituted for that purpose.\nNor can section 21 of the , vest such a power by necessary implication.\nIt is well settled that the rule of construction embodied in section 21 of the can apply to the provisions of a statute only where the subject matter, context and effect of such provisions are in no way inconsistent with such application.\nSo judged it is clear that that section cannot apply to section 10(1) of the .\nMinerva Mills Ltd. vs Their Workmen, ; , held inapplicable.\n1192 Strawboard Manufacturing Co. Ltd. vs Gutta Mill Workers ' Union, ; , explained.\nThe Textile Workers ' Union, Amritsar vs The State of Punjab and others, A. I. R. 1957 pun.\n255 and Hayendranath Bose vs Second Industrial Tribunal, , overruled.\nSouth Indian Estate Labour Relations Organisation vs The Stat

In [ ]:
#Number of rows
len(data)

2608

In [ ]:
#Split the data into train and validation dataset
from sklearn.model_selection import train_test_split

# Split into train and validation sets
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)


In [ ]:
# Reference : https://discuss.huggingface.co/t/convert-a-list-of-dictionaries-to-hugging-face-dataset-object/14670
from datasets import Dataset
train_data=Dataset.from_list(train_data)
eval_data=Dataset.from_list(val_data)

In [ ]:
train_data

Dataset({
    features: ['source', 'summary'],
    num_rows: 2347
})

In [ ]:
eval_data

Dataset({
    features: ['source', 'summary'],
    num_rows: 261
})

In [ ]:
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, load_from_disk
import numpy as np
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model_name = "drive/MyDrive/Abhishek/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def preprocess_data(data):
  #get all the dialogues
  inputs = [src for src in data['source']]
  #tokenize the dialogues
  model_inputs = tokenizer(inputs,  max_length=512, padding='max_length', truncation=True) #changed max_length to 512
  #tokenize the summaries
  with tokenizer.as_target_tokenizer():
    targets = tokenizer(data['summary'], max_length=128, padding='max_length', truncation=True) #changed max_length to 128

  #set labels
  model_inputs['labels'] = targets['input_ids']
  #return the tokenized data
  #input_ids, attention_mask and labels
  return model_inputs

In [ ]:
tokenize_data_train= train_data.map(preprocess_data, batched = True)

Map:   0%|          | 0/2347 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
tokenize_data_eval= eval_data.map(preprocess_data, batched = True)

Map:   0%|          | 0/261 [00:00<?, ? examples/s]

In [ ]:
!pip install datasets transformers
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq
import datasets
import numpy as np

# Define the data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# Define the compute_metric function
def compute_metrics(pred):
    labels_ids = pred.label_ids
    return {'loss': 0.5} # Return a dictionary with a 'loss' key


In [ ]:


args = Seq2SeqTrainingArguments(
    'case-summ', #save directory
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size= 2,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    eval_accumulation_steps=3,
    fp16=True #available only with CUDA
    )

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenize_data_train,
    eval_dataset=tokenize_data_eval,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:

trainer.train()

Epoch,Training Loss,Validation Loss
1,3.277800,0.500000
2,2.827000,0.500000
3,2.600700,0.500000


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_

TrainOutput(global_step=1761, training_loss=2.845611658914058, metrics={'train_runtime': 2305.5278, 'train_samples_per_second': 3.054, 'train_steps_per_second': 0.764, 'total_flos': 7629291751735296.0, 'train_loss': 2.845611658914058, 'epoch': 3.0})

In [ ]:
model.save_pretrained("drive/MyDrive/Abhishek/bart-large-cnn")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [ ]:
source = '''Appeal No. 80 of 1952. Appeal from the Judgment and Decree dated the 6th September, 1950, of the High Court of Judicature at Calcutta (Das Gupta and Lahiri JJ.) in Appellate Decree No. 318 of 1949 from the Judgment and Decree dated the 25th February, 1949.
of the Court of the District Judge of Zillah 24 Parganas in Title Appeal No. 8 of 1948 arising out of the Judgment and Decree dated the 10th October, 1947, of the Court of the Additional Subordinate Judge, 7th Court, Alipore.
M.C. Setalvad, Attorney General for India (Aurobindo Guha and Gobinda Mohan Roy, with him) for the appellant.
Atul Chandra Gupta (Bijan Behari Das Gupta, with him) for respondent No. 1 1953.
November 16.
The Judgment of the Court was delivered by MUKHERJEA J.
The facts giving rise to this appeal are, for the most part, uncontroverted and the dispute between the parties centres round the short point as to whether a contract for sale of land to which this litigation relates, was discharged and came to an end by reason of certain supervening circumstances which affected the performance of a material part of it.
To appreciate the merits of controversy, it will be necessary to give a brief narrative of the material facts.
The defendant company, which is the main respondent in this appeal, is the owner of a large tract of land situated, in the vicinity of the Dhakuria Lakes within Greater Calcutta.
The 313 company started a scheme for development of this land for residential purposes which was described as Lake Colony Scheme No. I and in furtherance of the scheme the entire area was divided into a large number of plots for the sale of which offers were invited from intending purchasers.
The company 's plan of work seemed to be, to enter into agree ments with different purchasers for sale of these plots of land and accept from them only a small portion of the con sideration money by way of earnest at the time of the agree ment.
The company undertook to construct the roads and, drains necessary for making the lands suitable for building and residential purposes and as soon as they were completed.
the purchaser would be called upon to complete the con veyance by payment of the balance of the consideration money.
Bejoy Krishna Roy, who was defendant No. 2 in the suit and figures as a pro forma respondent in this appeal, was one of such purchasers who entered into a contract with the company for purchase of a plot of land covered by the scheme.
His contract is dated the 5th of August, 1940, and he paid Rs. 101 as earnest money.
In the receipt granted by the vendor for this earnest money, the terms of the agree ment are thus set out: "Received with thanks from Babu Bejoy Krishna Roy of 28 Tollygunge Circular Road, Tollygunge, the sum of Rs. 101 (Rupees one hundred and one only) as earnest money having agreed to sell to him or his nominee 5 K. more or less in plot No. 76 on 20 and 30 ft.
Road in Premises No. Lake Colony Scheme No. 1, Southern Block at the average rate of Rs. 1,000 (Rupees one thousand only) per Cotta.
The conveyance must be completed within one month from the date of completion of roads on payment of the balance of the consideration money, time being deemed as the Essence of the Contract.
In case of default this agreement will be considered as cancelled with forfeiture of earnest money.
Mokarari Mourashi 314 Terms of payment:One third to be paid at the time of registration and the balance within six years bearing Rs. 6 per cent.
interest per annum".
On 30th November, 1941, the plaintiff appellant was made a nominee by the purchaser for purposes of the contract and although he brought the present suit in the character of a nominee, it has been held by the trial judge as well as by the lower appellate court, that he was really an assignee of Bejoy Krishna Roy in respect to the latter 's rights under the contract.
Some time before this date, there was an order passed by the Collector, 24 Parganas, on 12th of November, 1941 under section 79 of the Defence of India Rules, on the strength of which a portion of the land covered by the scheme was requisitioned for military purposes.
Another part of the land was requisitioned by the Government on 20th of December, 1941.
while a third order of requisition, which related to the balance of the land comprised in the scheme, was passed sometime later.
In November, 1943, the company addressed a letter to Bejoy Krishna Roy informing him of the requisitioning of the lands by the Government and stating inter alia that a considerable portion of the land appertaining to the scheme was taken possession of by the Government and there was no knowing how long the Government would retain possession of the same.
The constructs of the proposed roads and drains, therefore, could not be taken up during the continuance of the war and possibly for many years after its termination.
In these circumstances,, the company decided to treat the agreement for sale with the addressee as cancelled and give him the option of taking back the earnest money within one month from the receipt of the letter.
There was offer made in the alternative that in case the purchaser refused to treat the contract as cancelled, he could, if he liked, complete the conveyance within one month from the receipt of the letter by paying the balance of the consideration money and take the land in the condition in which it existed at that time, the company undertaking to construct the roads and the drains, as circumstances might permit, after the termination of the war.
315 The letter ended by saying that in the event of the addressee not accepting either of the two alternatives, the agreement would be deemed to be cancelled and the earnest money would stand forfeited.
This letter was handed over by Bejoy Krishna to his nominee, the plaintiff, and there was some correspondence after that, between the plaintiff on the one hand and the company on the other through their respective lawyers into the details of which it is not necessary to enter.
It is enough to state that the plaintiff refused to accept either of the two alternatives offered by the company and stated categorically that the latter was bound by the terms of the agreement from which it could not, in law, resile.
On 18th of January, 1946, the suit, out of which this appeal arises, was commenced by the plaintiff against the defendant company, to which Bejoy Krishna Roy was made a party defendant and the prayers in the plaint were for a two fold declaration, namely, (1) that the contract dated the 5th of August, 1940, between the first and the second defendant, or rather his nominee, the plaintiff, was still subsisting; and (2) that the plaintiff was entitled to get a conveyance executed and registered by the defendant on payment of the consideration money mentioned in the agreement and in the manner and under the conditions specified therein.
The suit was resisted by the defendant company who raised a large number of defences in answer to the plaintiff 's claim, most of which are not relevant for our present purpose.
The principal contentions raised on behalf of the defendant were that a suit of this description was not maintainable under section 42 of the Specific Relief Act and that the plaintiff had no locus standi to institute the suit.
The most material plea was that the contract of sale stood discharged by frustration as it became impossible by reason of the supervening events to perform a material part of it.
Bejoy Krishna Roy did not file any written statement and he was examined by the plaintiff as a witness on his behalf.
316 The trial judge by his judgment dated 10th October, 1.947, overruled all the pleas taken by the defendant and decreed the plaintiff 's suit.
An appeal taken by the defendant to the Court of the District Judge of 24 Parganas was dismissed on the 25th February, 1949, and the judgment of the trial court was affirmed.
The defendant company thereupon preferred a second appeal to the High Court which was heard by a Division Bench consisting 'of Das Gupta and Lahiri JJ.
The only question canvassed before the High Court was, whether the contract of sale was frustrated by reason of the requisition orders issued by the Government? The learned Judges answered this question in the affirmative in favour of the defendant and on that ground alone dismissed the plaintiff 's suit.
The plaintiff has now come before us on the strength of a certificate granted by the High Court under article 133(I)(c) of the Constitution of India.
The learned Attorney General, who appeared in support of the appeal, has put forward a three fold contention on behalf of his client.
He has contended in the first place that the doctrine of English law relating to frustration of contract, upon which the learned Judges of the High Court based their Decision has no application to India in view of the statutory provision contained in section 56 of the Indian Contract Act.
it is argued in the second place, that even if the English law Applies, it can have no application to contracts for sale of land and that is in fact the opinion expressed by the English ,judges themselves.
His third and the last argument is that on the admitted faacts and circumstances of this case there was no frustrating event which could be said to have taken away the basis of the contract or tendered its performance impossible in any sense of the word.
The first argument advanced by the learned AttorneyGeneral raises a somewhat debatable point regarding the true scope and effect of section 56 of the Indian Contract Act and to what extent, if any, it incorporates the English rule of frustration of contracts.
317 Section 56 occurs in Chapter IV of the Indian Contract Act which relates to performance of contracts and it purports to deal with one circumstances under which performance of a, contract is excused or dispensed with on the ground of the contract being void.
The section stands as follows: "An agreement to do an act impossible in itself is void.
A contract to do an act which after the contract is made, becomes impossible, or, by reason of some event which the promiser could not prevent, unlawful, becomes void when the act becomes impossible or unlawful.
Where one person has promised to do something which he knew, or, with reasonable diligence, might have known, and which the promisee did not know to be impossible or unlawful, such promisor must make compensation to such promisee for any loss which such promise sustains through the non performance of the promise".
The first_paragraph of the section lays down the law in the same way as in England.
It speaks of something which is impossible inherently or by its very nature, and no one can obviously be directed to an act.
The second paragraph enunciates the law relating to discharge of contract by reason of supervening impossibility or illegality of the act agreed to be done.
The wording of this paragraph is quite general, and though the illustrations attached to it are not at all happy, they cannotderogate from the general words used in the enactment.
This much is clear that the word "impossible" has not been used here in the sense of physical or literal impossibility.
The performance of an act may not be literally impossible but it may be impracticbale and useless from the point of view of the object and purpose which the parties had in view and if an untoward event or change of circumstances totally upset the very foundation upon which the parties rested their bargain, it can very well be said that the promisor L/B(D)2SCI 6(a) 318 found it impossible to do the act which he promised to do.
Although various theories have been propounded by the Judges and jurists in England regarding the juridical basis of the doctrine of frustration, yet the essential idea upon which the doctrine is based is that of impossibility of performance of the contract: in fact impossibility and frustration are often used as interchangeable expressions.
The changed circumstances, it is said, make the performance of the contract impossible and the parties are absolved from the further performance of it as they did not promise to perform an impossibility The parties shall be excused, as Lord Loreburn says(1), "if substantially the whole contract becomes impossible of performance or in other words impracticable by some cause for which neither was responsible,.
" In Joseph Constantine Steamship Line Limited vs Imperial Smelting Corporation Ltd.(2), Viscount Maugham obseryed that the "doctrine of frustration is only a special case of the discharge of contract by an impossibility of performance arising after the contract was made." Lord Porter agreed with this view and rested the doctrine on the same basis.
The question was considered and discussed by a Division Bench of the Nagpur High Court in Kesari Chand vs Governor General in Council(3) and it was held that the doctrine of frustration comes into play when a contract becomes impossi ble of performance, after it is made, on account of circum stances beyond the control of the parties.
The doctrine is a special case of impossibility and as such comes under section 56 of the Indian Contract Act.
We are in entire agreement with this view which is fortified by a recent pronouncement of this court in Ganga Saran vs Ram Charan(4), where Fazl (1) See Tamplin Steamship Co. Ltd. vs Anglo Mexican Petroleum Products Co. Ltd.[1916] 2 A.C. 397, 403.
(2) at 168.
(3) I.L.R. (4) ; at 52.
319 Ali J., in speaking about frustration, observed in his judgment as follows: "It seems necessary for us to emphasise that so far as the courts in this country are concerned, they must loot primarily to the law as embodied in sections 32 and 56 of the .
" We hold, therefore, that the doctrine of frustration is really an aspect or part of the law of discharge of contract by reason of supervening impossibility or illegality of the act agreed to be done and hence comes within the purview of section 56 of the .
It would be incorrect to say that section 56 of the Contract Act applies only to cases of physical impossibility and that where this section is not applicable, recourse can be had to the principles of English law on the subject of frustration.
It must be held also that to the extent that the deals with a particular subject, it is exhaustive upon the same and it is not permissible to import the principles of English law dehors these statutory provisions.
The decisions of the English courts possess only a persuasive value and may be helpful in showing how the courts in England have decided cases under circumstances similar to those which have come before our courts.
It seems necessary however to clear up some misconception which is likely to arise because of the complexities of the English law on the subject.
The law of frustration in England developed, as is well known, under the guise of reading implied terms into contracts.
The court implies a term or exception and treats that as part of the contract.
In the case of Taylor vs Caldwell(1), Blackburn J. first formulated the doctrine in its modern form.
The court there was dealing with a case where a music hall in which one of the contracting parties had agreed to give concerts on certain specified days was accidentally burnt by fire.
It was held that such a contract must be regarded "as subject to an implied condition that the parties shall be excused, in case, before breach, performance becomes impossible from perishing of the thing without (1) ; 320 default of.
the contractor.
" Again in Robinson vs Davison(1) there was a contract between the plaintiff and the defendant 's wife (as the agent of her husband) that she should play the piano at a concert to be given by the plaintifl on a specified day.
On the day in question she was unable to perform through illness.
The contract did not contain any term as to what was to be done in case of her being too ill to perform.
In an action against the defendant for breach of contract, it was held that the wife 's illness and the consequent incapacity excused her and that the contract was in its nature not absolute but conditional upon her being well enough to perform.
Bramwell B. pointed out in course of his judgment that in holding that the illness of the defendant incapaciated her from performing the agreement the court was not really engrafting a new term upon an express contract.
It was not that the obligation was absolute in the original agreement and a new condition was subsequently added to it; the whole question was whether the original contract was absolute or conditional and having regard to the terms of the bargain, it must be held to be conditional.
The English law passed through various stages of development since then and the principles enunciated in the various decided authorities cannot be said to be in any way uniform.
In many of the pronouncements of the highest courts in England the doctrine of frustration was held "to be a device by which the rules as to absolute contracts are reconciled with a special exception which justice demands"(2).
The court, it is said, cannot claim to exercise a dispensing power or to modify or alter contracts.
But when an unexpected event or change of circumstance occurs, the possibility of which the parties did not circumstance occurs, the possibility contract is taken to be not what the parties actual intended, but what they as fair and reasonable men would presumably have intended and agreed upon, if having such possibility in view they had made express provsion as to their rights and liabilities in the event of such occurrence(1).
As Loard Wright (1) ; (2) Vide Hirji Mulji vs Cheong Yue Steamship Co. Ltd. at 510.
(3) Vide Dahl vs Nelson, Donkinand Co. (1881) 6 App.
38 at 59.
321 observed in Joseph Constantine Steamship Co. vs Imperial Smelting Corporation Ltd.(1).
"In ascertaining the meaning of the contract and its application to the actual occurrences, the court has to decide, not what the parties actually intended but what as reasonable men they should have intended.
The court personifies for this purpose the reasonable man.
" Lord Wright clarified the position still further in the later case of Denny, Mott and Dickson Ltd. vs James B. Fraser & Co. Ltd.(1), where he made the following observations: "Though it has been constantly said by high authority, including Lord Sumner, that the explanation of the rule is to be found in the theory that it depends on an implied con dition of the contract, that is really no explanation.
It only pushes back the problem a single stage.
It leaves the question what is the reason for implying a term.
Nor can I reconcile that theory with the view that the result does not depend on what the parties might, or would, as hard bargainers, have agreed.
The doctrine is invented by the court in order to supplement the defects of the actual contract.
To my mind the theory of the implied condition is not really consistent with the true theory of frustration.
It has never been acted on by the court as a ground of decision, but is merely stated as a theoretical explanation.
" In the recent case of British Movietonews Ltd. vs London and District Cinemas Ltd.(1), Denning L. J. in the Court of Appeal took the view expressed by Lord Wright as stated above as meaning that "the court really exercises a qualifying power a power to qualify the absolute., literal or wide terms of the contract in order to do what is just and reasonable in the new situation".
"The day is gone," (1) at 185.
(2) ; at 275.
(3) L/ B(D) 2SCI 7 322 the learned Judge went on to say, "when we can excuse an unforeseen injustice by saying to the sufferer 'it is your own folly, you ought not to have passed that form of words.
You ought to have put in a clause to protect yourself '.
We no longer credit a party with the foresight of a Prophet or his lawyer with the draftsmanship of a Chalmers.
We realise that they have their limitations and make allowances accor dingly.
It is better thus.
The old maxim reminds us that he who clings to the letter clings to the dry and barren shell and misses the truth and substance of the matter.
We have of late paid heed to this warning, and we must pay like heed now.
" This decision of the Court of Appeal was reversed by the House of Lords and Viscount Simon in course of his judgment expressed disapproval of the way in which the law was stated by Denning L.J.
It was held that there was no change in the law as a result of which the courts could exercise a wider power in this regard than they used to do previously.
"The principle remains the same", thus observed his Lordship. "Particular applications of it may greatly vary and theoretical lawyers may debate whether the rule should be regarded as arising from implied term or because the basis of the contract no longer exists.
In any view, it is a question of construction as Lord Wright pointed out in Constantine 's case and as has been repeatedly asserted by other masters of law.
"(1) These differences in the way of formulating legal theories really do not concern us so long as we have a statutory provision in the .
In deciding cases in India the only doctrine that we have to go by is that of supervening impossibility or illegality as laid down in section 56 of the Contract Act taking the word "Impossible" in its practical and not literal sense.
It must be borne in mind, however, that section 56 lays down a rule of positive law and does not leave the matter to be determined according to the intention of the parties.
(1) at 184.
323 In the latest decision of the House of Lords referred to above, the Lord Chancellor puts the whole doctrine upon the principle of construction.
But the question of construction may manifest itself in two totally different ways.
In one class of cases the question may simply be, as to what the parties themselves had actually intended and whether or not there as a condition in the contract itself, express or implied, which operated, according to the agreement of the Parties themselves to release them from their obligations; this would be a question of construction pure and simple and the ordinary rules of construction would have to be applied to find out what the real intention of the parties was.
According to the , a promise may be express or implied(1).
In cases, therefore, where the court gathers as a matter of construction that the contract itself contained impliedly or expressly a term, according to which it would stand discharged on the happening of certain circumstances the dissolution on of the contract would take place under the terms of the contract itself and such cases would be outside the purview of section 56 altogether.
Although in English law these cases are treated as cases of frustration, in India they would be dealt with under section 32 of the which deals with contingent contracts or similar other provisions contained in the Act.
In the large majority of cases however the doctrine of frustration is applied not on the ground that the parties themselves agreed to an implied term which operated to release them from the performance of the contract.
The relief is given by the court on the ground of subsequent impossibility when it finds that the whole purpose or basis of a contract was frustrated by the intrusion or occurrence of an unexpected event or change of circumstances which was beyond what was contemplated by the parties at the time when they entered into the agreement.
Here there is no question of finding out an implied term agreed to by the parties em bodying a provision for discharge, because the parties did not think about the matter at all nor could possibly have any intention regarding it.
When such an event or change of (1) Vide section 9.
L/B(D)2SCI 7(a) 324 circumstance occurs which is so fundamental as to be re garded by law as striking at the root of the contract as a whole, it is the court which can pronounce the contract to be frustrated and at an end.
The court undoubtedly has to examine the contract and the circumstances under which it was made.
The belief, knowledge and intention of the parties are evidence, but evidence only on which the court has to form its own conclusion whether the changed cir cumstances destroyed altogether the basis of the adventure and its underlying object(1).
This may be called a rule of construction by English Judges but it is certainly not a, principle of giving effect to the intention of the parties which underlies all rules of construction.
This is really a rule of positive law and as such comes within the purview of section 56 of the .
It must be pointed out here that if the parties do con template the possibility of an intervening circumstance which might affect the performance of the contract, but expressly stipulate that the contract would stand despite such circumstances, there can be no case of frustration because the basis of the contract being to demand performance despite the happening of a particular event, it cannot disappear when that event happens.
As Lord Atkinson said in Matthey vs Curling(1), "a person who expressly contracts absolutely to do a thing not naturally impossible is not excused for nonperformance because of being prevented by the act of God or the King 's enemies. . or vis major".
This being the legal position, a contention in the extreme form that the doctrine of frustration as recognised in English law does no come at all within the purview of section 56 of the cannot be accepted.
The second contention raised by the Attorney General can be disposed of in few words.
It is true that in England the judicial opinion generally expressed is, that the doctrine of frustration does not operate in the case of contracts for (1) Vide Morgan vs Manser (1947] 2 AU E.R. 666.
(2) at 234.
325 sale of land(1).
But the reason underlying this view is that under the English law as soon as there is a concluded contract by A to sell land to B at certain price, B becomes in equity, the owner of the land, subject to his obligation to pay the purchase money '.
On the other hand, A in spite of his having the legal estate holds the same in trust for the purchaser and whatever rights he still retains in the land are referable to his right to recover and receive the purchase money.
The rule of frustration can only put an end to purely contractual obligations, but it cannot destroy an estate in land which has already accrued in favour of a contracting party.
According to the Indian law, which is embodied in section 54 of the Transfer of Property Act, a contract for sale of land does not of itself create any interest in the property which is the subject matter of the contract.
The obligations of the parties to a contract for sale of land are, therefore, the same as in other ordinary contracts and consequendy there is no conceivable reason why the doctrine of frustration should not be applicable to contracts for sale of land in India.
This contention of the Attorney General must, therefore, fail.
We now come to the last and most important point in this case which raises the question as to whether, as a result of the requisition orders, under which the lands comprised in the development scheme of the defendant company were requisitioned by Government, the contract of sale between the defendant company and the plaintiff 's predecessor stood dissolved by frustration or in other words became impossible of performance.
It is well settled and not disputed before us that if and when there is frustration the dissolution of the contract occurs automatically.
It does not depend, as does rescission of a contract on the ground of repudiation or breach, or on the choice or election of either party.
It depends on the effect (1) Vida Billington Estates Co. vs Stonfield Estate Ltd. [1952] 1 All E.R.853.
326 of what has actually happened on the possibility of performing the contrat (1).
What happens generally in such cases and has happened here is that one party claims that the contract has been frustrated while the other party denies it.
The issue has got to be decided by the court "ex post facto, on the actual circumstances of the case"(2).
We will now proceed to examine the nature and terms of the contract before us and the circumstances under which it was entered into to determine whether or not the disturbing element,which is allowed to have happened here, has substantially prevented the performance of the contract as a whole.
It may be stated at the outset that the.
contract before us cannot be looked upon as an ordinary contract for sale and purchase of a piece of land; it is an integral part of a development scheme started by the defendant company and is one of the many contracts that have been entered into by a large number of persons with the company.
The object of the company was undoubtedly to develop a fairly extensive area which was still undeveloped and make it usable for residential purposes by making roads and constructing drains through it.
The purchaser.
on the other hand, wanted the land in regard to which he entered into the contract to be developed and make ready for building purposes before he could be called upon to complete the purchase.
The most material thing which deserves notice is, that there is abso lutely no time limit within which.
the roads and drains are to be made.
The learned District Judge of Alipore, who heard the appeal, from the trial court 's judgment found it as a fact, on the evidence in the record, that there was not an understanding between the parties on this point.
As a matter of fact, the first requisition order was passed nearly 15 months after the contract was made and apparently no work was done by the defendant company in the meantime.
Another important thing that requires notice in this con (1) Per Lord Wright in Denny, Mott and Dicksom Ltd. vs Jameso B. Fraser and Co., Ltd. ; , 274, (2) Ibid.
327 nection is that the war was already on, when the parties entered into the contract.
Requisition orders for taking temporary possession of lands for war purposes were normal events during this period.
Apart from requisition orders there were other difficulties in doing construction work at that time because of the scarcity of materials and the various restrictions which the Government had imposed in respect of them.
That there were certain risks and difficulties involved in carrying on operations like these, could not but be in the contemplation of the parties at the time when they entered into the contract, and that is probably the reason why no definite time limit was mentioned in the contract within which the roads and drains are to be completed.
This was left entirely to the convenience of the company and as at matter of fact the purchaser did not feel concerned about it.
It is against this background that we are to consider to what extent the passing of the requisition orders affected the performance of the contract in the present case.
The company, it must be admitted, bad not commenced the development work when the requisition order was passed in November, 1941.
There was no question, therefore, of any work or service being interrupted for an indefinite period of time.
Undoubtedly the commencement of the work was delayed but was the delay going to be so great and of such a character that it would totally upset the basis of the bargain and comercial object which the parties had in view? The requisition orders, it must be remembered, were ' by their very nature, of a temporary character and the requisitioning authorities could, in law, occupy the position of a licensee in regard to the requisitioned property.
The order might continue during the whole period of the war and even for some time after that or it could have been withdrawn before the war terminated.
If there was a definite time limit agreed to by the parties within which the construction work was to be finished, it could be said with perfect propriety that delay for an indefinite period would 328 make the performance of the contract impossible within the specified time and this would seriously affect the object and purpose of the venture.
But when there is no time limit whatsoever in the contract, nor even an understanding bet ween the parties on that point and when during the war the parties could naturally anticipate restrictions of various kinds which would make the carrying on of these operations more tardy and difficult than in times of peace, we do not think that the order of requisition affected the fundamental basis upon which the agreement rested or struck at the roots of the adventure.
The learned Judges of the High Court in deciding the case against the plaintiff relied entirely on the time factor.
It is true that the parties could not contemplate an absolutely unlimited period of time to fulfil their contract.
They might certainly have in mind a period of time which was reasonable having regard to the nature and magnitude of the work to be done as well as the conditions of war prevailing at that time.
Das Gupta, J., who delivered the judgment of the High Court, says first of all that the company had in contemplation a period of time not much exceeding 2 or 3 years as the time for performance of the contract; the purchaser also had the same period of time in contemplation.
The learned Judge records his finding on the point in the following words: "My conclusion on a consideration of the surrounding circumstances of the contract is that the parties contemplated that the roads and drains would be constructed and the conveyance would be completed in the not distant future.
" This finding is inconclusive and goes contrary to what has been held by the District Judge who was undoubtedly the last court of facts.
In our opinion, having regard to the nature and terms of the contract, the actual existence of war conditions at the time when it was entered into, the extent of the work involved in the development scheme and last though not the least the total absence of any definite period of time agreed to by the parties within which the work was 329 to be completed, it cannot be said that the requisition order vitally affected the contract or made its performance impossible.
Mr. Gupta, who appeared for the respondent company.
put forward an alternative argument that even if the performance of the contract was not made impossible.
it certainly became illegal as a result of the requisition order and con sequently the contract became void under section 56 of the as soon as the requisition order was made.
In support of his contention the learned counsel placed reliance upon certain provisions of the Defence of India Rules and also upon illustration (d) to section 56 of the Contract Act.
All that the Defence Regulations show is that the violation of a requisition order could be punished as a criminal offence.
But no matter in whichever way the requisition order could be enforced, in substance it did nothing else but impose a prohibition on the use of the land during the period that it remained in force.
The effect of such prohibition on the performance of the contract, we have discussed above, and we do not think that the mere fact that the requisition order was capable of being enforced by a criminal sanction made any difference in this respect.
In any view this question was not raised in any of the courts below and has not been indicated even in the respondent 's statement of the case.
We do not think that it would be proper to allow this question to be raised for the first time before us, as it requires consideration of the different provisions of the Defence of India Act and also of the implication of illustration (d) appended to section 56 of the Contract Act.
In our opinion, the events which have happened here cannot be said to have made the performance of the contract impossible and the contract has not been frustrated at all.
The result is that the appeal is allowed, the judgment and decree of the High Court of Calcutta are set aside and those of the courts below restored.
The plaintiff will have his costs in all the courts.
Appeal allowed.
Agent for the respondent No. I : R. R. Biswas.'''

In [ ]:
#Better way

# Tokenize the conversation
inputs = tokenizer(source, return_tensors="pt", truncation=True, max_length=1024) # Added truncation and max_length for the tokenizer

# Ensure inputs are on the same device as the model (assuming model is on CUDA)
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# Print the maximum token ID in the input
print(f"Maximum token ID in input: {inputs['input_ids'].max()}")

# Print the vocabulary size of the model
print(f"Model vocabulary size: {model.config.vocab_size}")

# Generate summary
summary_ids = model.generate(inputs["input_ids"], max_length=1024) # Adjust max_length as needed

# Decode the summary
summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Maximum token ID in input: 50118
Model vocabulary size: 50264


In [ ]:
# Tokenize the conversation
inputs = tokenizer(source, return_tensors="pt", truncation=True, max_length=1024)

# Ensure inputs are on the same device as the model
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# Generate summary with adjusted parameters
summary_ids = model.generate(
    inputs["input_ids"],
    max_length=256,  # Increased max_length
    num_beams=5  ,    # Try beam search with a wider beam
    no_repeat_ngram_size=2,  # Prevent repetitions
    early_stopping=False,     # Disable early stopping if it was enabled
)

# Decode the summary
summary = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

In [ ]:
print(summary)

The appellant, a private limited company, was carrying on business in the manufacture and sale of paper and paper board.
It was assessed to income tax under the Income tax Act, 1961 for the assessment years 1961 62, 1962 63, 1963 64, 1964 65, 1965 66, 1966 67, 1967 68, 1968 69, 1969 68 and 1969 69 69 in respect of the year ending 31.12.1966, which was the last year in which the assessable income was calculated and the tax payable on the income received by the company for that year was included in its assessment under section 3(1)(a) (b) and (c).
The Income Tax Officer held that the assessee was not liable to pay tax as it did not fall within the definition of 'income ' contained in sub section (1) of section 5(2)


In [ ]:
!pip install transformers
from transformers import AutoTokenizer

# Example: Load a pre-trained tokenizer
model_checkpoint = "t5-small"  # Replace with your desired model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

from torch.utils.data import Dataset, DataLoader

class SummaryDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=1024):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        source = self.data[idx]["source"]
        target = self.data[idx]["summary"]

        inputs = self.tokenizer(
            source,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )
        labels = self.tokenizer(
            target,
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt",
        )

        # Prepare data for the model
        input_ids = inputs.input_ids.squeeze()
        attention_mask = inputs.attention_mask.squeeze()
        labels = labels.input_ids.squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels,
        }

# Example data (replace with your actual data)
data=list()
data = load_data(data)

# Create dataset and dataloader
dataset = SummaryDataset(data, tokenizer)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Define your model and tokenizer
model_name = "facebook/bart-base"  # Example model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Define your dataset and dataloader
# ... (replace with your data loading logic)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# Fine-tuning loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

num_epochs=3
for epoch in range(num_epochs):
  model.train()
  for batch in dataloader:
    # Move batch data to device
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    # Forward pass
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # Evaluation (optional)
  # ...

# Save the fine-tuned model
model.save_pretrained("content/drive/MyDrive/finetuned_model_t5")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
